This notebook is a playground for generating pixelwise weights which can be used to get better-defined edges with the semantic segmentation task of unet_dev

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

import os

from skimage.measure import label

from scipy.ndimage import morphology


In [ ]:
#get some masks to play with...
DataDir = './data/pericardial/wsx_20200221/'

Y = np.load(os.path.join(DataDir,'Y.npy')).astype('float')

#show an example
m,ypx,xpx = Y.shape
plt.imshow(Y[np.random.randint(m),:,:])
plt.title(str(m) + ' masks of ' + str(xpx) + '*' + str(ypx) + ' pixels')

In [ ]:
#some code stolen shamelessly from the internet, with the original u-net weighting function (Ronneberger et al 2015, code from https://stackoverflow.com/questions/50255438/pixel-wise-loss-weight-for-image-segmentation-in-keras)

from skimage.measure import label

from scipy.ndimage.morphology import distance_transform_edt


def unet_weight_map(y, wc=None, w0 = 10, sigma = 5):
    """
    Generate weight maps as specified in the U-Net paper
    for boolean mask.

    "U-Net: Convolutional Networks for Biomedical Image Segmentation"
    https://arxiv.org/pdf/1505.04597.pdf

    Parameters
    ----------
    mask: Numpy array
        2D array of shape (image_height, image_width) representing binary mask
        of objects.
    wc: dict
        Dictionary of weight classes.
    w0: int
        Border weight parameter.
    sigma: int
        Border width parameter.

    Returns
    -------
    Numpy array
        Training weights. A 2D array of (image_height, image_width).
    """

    labels = label(y)
    
    no_labels = labels == 0
    label_ids = sorted(np.unique(labels))[1:]

    if len(label_ids) > 1:
        distances = np.zeros((y.shape[0], y.shape[1], len(label_ids)))

        for i, label_id in enumerate(label_ids):
            distances[:,:,i] = distance_transform_edt(labels != label_id)

        distances = np.sort(distances, axis=2)
        d1 = distances[:,:,0]
        d2 = distances[:,:,1]
        w = w0 * np.exp(-1/2*((d1 + d2) / sigma)**2) * no_labels
    else:
        w = np.zeros_like(y)
    if wc:
        class_weights = np.zeros_like(y)
        for k, v in wc.items():
            class_weights[y == k] = v
        w = w + class_weights
    return w

In [ ]:
def show_mask_weight(mask,weight):
    
    plt.figure(figsize = (10,5))
    
    plt.subplot(1,2,1)
    
    plt.imshow(mask)
    
    plt.title('mask')
    
    plt.xticks([])
    plt.yticks([])

    
    plt.subplot(1,2,2)
    
    plt.imshow(weight)
    
    plt.title('weight')
    
    plt.xticks([])
    plt.yticks([])


In [ ]:
eg = Y[0,:,:]

weight = unet_weight_map(eg,
                         wc = {0:1,1:50}, #approximately class weights
                         w0 = 10000,
                         sigma = 100,
                        )

show_mask_weight(eg, weight)

So, this demonstrates fairly clearly that this is totally useless. What I want is a weighting which is relative to BORDERS rather than objects.

In [ ]:
def nearest_border_map(mask):
    
    '''returns a pixelwise measure of the distance to nearest border pixel. obvs use the euclidean one.'''
    
    #distance to background for foreground pixels. 
    fg = morphology.distance_transform_edt(mask)
    
    #distance to foreground for background pixels
    bg = morphology.distance_transform_edt(1-mask)
    
    distance = fg+bg
    
    return distance
    
dist = nearest_border_map(eg)

show_mask_weight(eg,dist) 

So, this looks pretty but is obviously of the incorrect polarity. a basic think is to take exp(y) for this image?

In [ ]:
show_mask_weight(eg,np.exp(-nearest_border_map(eg)))

OK, but it's a bit too crispy.... need to prescale the distances.

In [ ]:
show_mask_weight(eg,np.exp(-0.1*nearest_border_map(eg)))

This has a number of desirable properties:
 - represents a useful thing
 - easily rescaled
 - strictly >=0 
 - never undefined (like using reciprocals)
 
HOWEVER, it is also annoying because of hyperparameters:
 - sigma, which scales distances prior to exponentiation
 - will also need rescaling afterwards if we want to combine it with other stuff.

Wrap it up in a nice function... 

In [ ]:
def closeness_to_border(mask,sigma=20):
    
    #distance to background for foreground pixels. 
    fg = morphology.distance_transform_edt(mask)
    
    #distance to foreground for background pixels
    bg = morphology.distance_transform_edt(1-mask)
    
    distance = fg+bg
    
    closeness = np.exp(-distance/sigma)
    
    #ensure mean of 1
    closeness /= np.mean(closeness)
    
    return closeness

In [ ]:
show_mask_weight(eg,closeness_to_border(eg))

In [ ]:
plt.hist(closeness_to_border(eg).flatten())

However, perhaps this is not the correct approach. ALSO, this leads to some weights very close to zero... I think that a better approach might be to use the distance to a foreground pixel, with the maximum height of the exponential defined by the class imbalance? 

In [ ]:
def closeness_to_foreground_balanced(mask,sigma=20):
    
    distance = morphology.distance_transform_edt(1-mask)
    
    closeness = np.exp(-distance/sigma) #which will be between 0 and 1
    
    #get class imbalance, assuming that there are less +ve pixels
    imbalance = mask.size/mask.sum()
    
#     #rescale to be between 1 and imbalance: i.e. the fg pixels are imbalance, the far bg are 1, and scaling is between these 2 values.
    closeness = closeness*(imbalance-1)+1
    
    #rescale for mean of 1
    closeness /= np.mean(closeness)
    
    return closeness
    

In [ ]:
show_mask_weight(eg,closeness_to_foreground_balanced(eg,sigma=20))